In [1]:
#BP神经网络
%matplotlib qt5
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt  
from mpl_toolkits.mplot3d import Axes3D  

Using TensorFlow backend.


In [2]:
#训练数据
X = []
Y = []
Z = []
path = '/Users/liuyingjie/Downloads/data/multi/1/'
filenames = os.listdir(path)
indexs = []
for filename in filenames:
    if filename.startswith("."):
        continue
    indexs.append(int(filename))
indexs.sort()
#print(indexs)
for index in indexs:
    file_path = path + str(index)
    with open(file_path,'r',encoding='utf8') as f:
        lines=f.readlines()
    for num in range(102, 122):
        #print(lines[num])
        temps = lines[num].split()
        X.append(float(temps[2])/100)
        Y.append(float(temps[3])/100)
    for line in lines:
        temps = line.split()
        size = len(temps)
        if(size == 11 and temps[0]=='10000000'):
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
        if(size == 16 and temps[0] == '10000000'):
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
            Z.append(int(float(temps[11])*37000000000))
            
          

In [3]:
#预测数据
Point1=[90,110]#预测场左上角坐标
Point2=[110,90]#预测场右下角坐标
X_interval=0.3
Y_interval=-0.3
X_p = []
Y_p = []
X_start=Point1[0]-X_interval
Y_start=Point1[1]-Y_interval
while abs(Point2[0] -X_start)>=abs(X_interval)or abs(Point2[1]-Y_start)>=abs(Y_interval):
    X_start=Point1[0]-X_interval
    Y_start+=Y_interval
    while abs(Point2[0] -X_start)>=abs(X_interval):
        X_start+=X_interval
        X_p.append(X_start)
        Y_p.append(Y_start)
#print(X_p)
#print(Y_p)
#print(len(X_p))
#print(len(Y_p))

In [4]:
X_pred = scale(np.array(X_p))
Y_pred = scale(np.array(Y_p))
X_pred_ = np.array([X_pred,Y_pred])
X = scale(np.array(X))
Y = scale(np.array(Y))
X_ = np.array([X,Y])
Y_ = scale(np.array(Z))

In [5]:
from sklearn.ensemble import GradientBoostingRegressor 
params = {
'n_estimators': 400, 
'max_depth': 8,
'learning_rate': 0.06,
 'loss': 'ls',
 'subsample':0.8
 }
model=GradientBoostingRegressor(**params)
model.fit(X_.T, Y_.T)
# 对测试集进行预测
Y_pred = model.predict(X_pred_.T)


def getDistance(num1,num2):
    return pow(abs(num1[0]-num2[0]),2)+pow(abs(num1[1]-num2[1]),2)


ans=[]
predict=[]
xLength=int(abs(Point2[0]-Point1[0])/abs(X_interval))
yLength=int(abs(Point2[1]-Point1[1])/abs(Y_interval))
i=0
while i <len(X_p):
    j=0
    while j<=xLength:
        r=[X_p[i],Y_p[i],Y_pred[i]]
        ans.append(r)
        i+=1
        j+=1

ans.sort(key=lambda k:k[2],reverse = True)

#print(ans)

pre = []
ansLen, ansX, ansY = 0, 0, 0
for i in range (len(ans)):
    if len(pre) == 4:
        break
    if ans[i][2] == ans[i+1][2]:
        continue
    ansLen += 1
    ansX += ans[i][0]
    ansY += ans[i][1]
    ansX /= ansLen
    ansY /= ansLen
    prePoint = [ansX,ansY,ans[i][2]]
    if len(pre) == 0 :
        pre.append(prePoint)
        continue
    if len(pre) == 1 and getDistance(pre[0], prePoint) > 3:
        pre.append(prePoint)
        continue
    if len(pre) == 2 and getDistance(pre[0], prePoint) > 3 and getDistance(pre[1], prePoint) > 3:
        pre.append(prePoint)
        continue
    if len(pre) == 3 and getDistance(pre[0], prePoint) > 3 and getDistance(pre[1], prePoint) > 3 and getDistance(pre[2], prePoint) > 3:
        pre.append(prePoint)
        continue
    ansLen, ansX, ansY = 0, 0, 0
print(pre)

[[102.89999999999988, 102.50000000000007, 6.977431927268721], [106.79999999999984, 98.9000000000001, 6.974656342341259], [93.59999999999997, 106.10000000000004, 6.927399069940098], [97.79999999999993, 93.50000000000016, 6.3330619696909345]]


In [8]:
ax = plt.figure().add_subplot(111, projection = '3d')  
ax.scatter(X_pred_[0], X_pred_[1], Y_pred, c = 'b', marker = 'o') #点为红色三角形  
ax.set_xlabel('X Label')  
ax.set_ylabel('Y Label')  
ax.set_zlabel('Z Label')  
plt.show()